### **preparing enviroment**

installing modules

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

setting enviroment

In [0]:
%tensorflow_version 1.x
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

test is everything correct installed

In [0]:
!python object_detection/builders/model_builder_test.py

### **preparing folders**

deleting and creating *data* directory

In [0]:
%cd /content/
DATA_DIR = 'data/'
!rm -rf {DATA_DIR}
os.makedirs(DATA_DIR, exist_ok=True)

downloading model

In [0]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL = 'ssd_inception_v2_coco_2018_01_28'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/data'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

copying training files from google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

MODEL_DIR = '/content/drive/My\ Drive/TF9/training/'

!cp /content/drive/My\ Drive/TF9/train.record /content/data/train.record
!cp /content/drive/My\ Drive/TF9/val.record /content/data/val.record
!cp /content/drive/My\ Drive/TF9/label_map.pbtxt /content/data/label_map.pbtxt
!cp /content/drive/My\ Drive/TF9/pipeline.config /content/data/pipeline.config

### **running TensorBoard**

In [0]:
%cd /content/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [0]:
LOG_DIR = MODEL_DIR
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### **training the model**

In [0]:
PIPELINE_FNAME = '/content/data/pipeline.config'
# check NUM_STEPS in pipeline.config 
NUM_STEPS = 200000
NUM_EVAL_STEPS = 5

In [0]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={PIPELINE_FNAME} \
    --model_dir={MODEL_DIR} \
    --alsologtostderr \
    --num_train_steps={NUM_STEPS} \
    --num_eval_steps={NUM_EVAL_STEPS}

### **exporting the frozen graph**

In [0]:
%cd /content/
!mkdir training
!cp -r /content/drive/My\ Drive/TF9/training/ /content/

In [0]:
%cd /content/

import re
import numpy as np

OUTPUT_DIR = '/content/output'
MODEL_DIR = '/content/training/'

lst = os.listdir(MODEL_DIR)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(MODEL_DIR, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={PIPELINE_FNAME} \
    --output_directory={OUTPUT_DIR} \
    --trained_checkpoint_prefix={last_model_path}

copy to google drive

In [0]:
!cp /content/output/frozen_inference_graph.pb /content/drive/My\ Drive/TF9/frozen_inference_graph.pb

### **inference**

copying frozen graph from google drive **if needed**

In [0]:
%cd /content
!mkdir output
!cp /content/drive/My\ Drive/TF9/frozen_inference_graph.pb /content/output/frozen_inference_graph.pb
assert os.path.isdir('/content/output')

initializing

In [0]:
import os
import glob

PATH_TO_CKPT = '/content/output/frozen_inference_graph.pb'
PATH_TO_LABELS = '/content/data/label_map.pbtxt'
PATH_TO_TEST_IMAGES_DIR = '/content/drive/My Drive/TF9/input/'
PATH_TO_RESULT_IMAGES_DIR = '/content/drive/My Drive/TF9/result/'

assert os.path.isfile(PATH_TO_CKPT)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)
JPG_PATHS = [ os.path.basename(path) for path in TEST_IMAGE_PATHS ]
RESULT_IMAGE_PATHS = [ os.path.join(PATH_TO_RESULT_IMAGES_DIR, jpg_path) for jpg_path in JPG_PATHS ]

object detection and export processed photos

In [0]:

%cd /content/models/research/object_detection

NUM_CLASSES = 4

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import glob
import argparse

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

# %matplotlib inline

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)


with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.

        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for image_path, result_path in zip(TEST_IMAGE_PATHS, RESULT_IMAGE_PATHS):
            
            image_np = cv2.imread(image_path, 1)
            
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
            
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)

            cv2.imwrite(result_path, image_np)
        print('done')